In [2]:
!nvidia-smi

Fri Aug  4 23:43:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 76.9 MB/s eta 0:00:0

In [4]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.2 MB/s eta 0:00:00
Found existing installation: transformers 4.31.0
Uninstalling transformers-4.31.0:
  Successfully uninstalled transformers-4.31.0
Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Successfully uninstalled accelerate-0.21.0
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)


# Import required packages

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline, set_seed
from datasets import load_dataset, load_from_disk,load_metric

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Check device

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Load Pretrained Tokenizer and Text Summarization Model on "google/pegasus-cnn_dailymail" dataset using Transformers

In [7]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
## Save model
model_pegasus.save_pretrained("/content/drive/MyDrive/Text-Summarizer/model.pkl")
tokenizer.save_pretrained("/content/drive/MyDrive/Text-Summarizer/tokenizer.pkl")

('/content/drive/MyDrive/Text-Summarizer/tokenizer.pkl/tokenizer_config.json',
 '/content/drive/MyDrive/Text-Summarizer/tokenizer.pkl/special_tokens_map.json',
 '/content/drive/MyDrive/Text-Summarizer/tokenizer.pkl/spiece.model',
 '/content/drive/MyDrive/Text-Summarizer/tokenizer.pkl/added_tokens.json',
 '/content/drive/MyDrive/Text-Summarizer/tokenizer.pkl/tokenizer.json')

In [9]:
News_Article = """
Penal Colony No 6 is 240km (150 miles) east of Moscow. Navalny, the Kremlin's most vocal critic, is already serving a nine-year term there for parole violations, fraud and contempt of court: charges widely seen as politically motivated.

His time behind bars looks set to be extended.

In June, a hall in the penal colony was turned into a makeshift courtroom and Navalny was back in the dock. The charges against him this time include creating an extremist organisation and financing extremist activities.

Russian state prosecutors have called for a 20-year prison sentence for Navalny in an even more restrictive "special regime colony". Such prisons are normally reserved for Russia's most dangerous criminals.

In a message posted on social media ahead of the verdict, which is due on Friday, Navalny expressed his belief that he would be handed "a big [prison] term".
We were among a group of journalists allowed briefly into the penal colony for the start of the trial. In a "press room" we could view proceedings on a video screen.

Navalny had clearly lost a lot of weight in prison. But he was defiant as he railed against the judge and his trial behind bars.

On paper, it is a Moscow court that is hearing the case. But the decision to hold a remote trial in a prison suggests that the Russian authorities wanted to avoid the publicity that would inevitably come with transporting Navalny to the Russian capital.

The picture was not on the screen for long. An hour and a half into the trial, the prosecutor demanded the closing of proceedings to press and public. The judge agreed. The video feed was cut.

That has made it difficult to follow what has been happening in this trial.

When Navalny delivered his final statement two weeks ago, there was no video or audio recording of his words. But the text of his speech criticising the Russian authorities and the war in Ukraine was made public. His supporters, including Russian actors and musicians in exile, read it out and posted online.

Even when this case is over, there may be more to follow. Navalny says that investigators have told him to expect another trial: on terrorism charges.
"""

In [ ]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}


pipe = pipeline("summarization", model='/content/drive/MyDrive/Text-Summarizer/model.pkl',tokenizer='/content/drive/MyDrive/Text-Summarizer/tokenizer.pkl')

##
print("News Article:")
print(News_Article)

print("\n News Summary:")
print(pipe(News_Article, **gen_kwargs)[0])

News Article:

Penal Colony No 6 is 240km (150 miles) east of Moscow. Navalny, the Kremlin's most vocal critic, is already serving a nine-year term there for parole violations, fraud and contempt of court: charges widely seen as politically motivated.

His time behind bars looks set to be extended.

In June, a hall in the penal colony was turned into a makeshift courtroom and Navalny was back in the dock. The charges against him this time include creating an extremist organisation and financing extremist activities.

Russian state prosecutors have called for a 20-year prison sentence for Navalny in an even more restrictive "special regime colony". Such prisons are normally reserved for Russia's most dangerous criminals.

In a message posted on social media ahead of the verdict, which is due on Friday, Navalny expressed his belief that he would be handed "a big [prison] term".
We were among a group of journalists allowed briefly into the penal colony for the start of the trial. In a "pr